# 배치정규화
- 드롭아웃과 마찬가지로 신경망 계층으로 구현되어 작동할 수 있는 정규화 기법이다. 
- 드롭아웃처럼 모델의 모드 전환에 따라 다르게 동작한다는 점도 같다. 
- 하지만 배치정규화 기법의 경우 학습 속도를 비약적으로 향상시킬 수 있을 뿐만 아니라 일반화 성능까지 대폭 개선할 수 있는 훌륭한 방법이다.

## 공변량 변화 문제
- 배치정규화는 공변량 변화(covariate shift)문제를 해결하고자 제안되었습니다.
- 공변량 변화 문제가 무엇인지 알아보도록 하겠다.
- ![](../img/%EA%B3%B5%EB%B3%80%EB%9F%89%EB%B3%80%ED%99%94.png)

- 어떤 입력이 주어졌을 때 신경망은 이에 대한 모든 모델의 출력을 반환할 것이다.
- 그러면 이를 활용하여 손실을 계산하고 곧이어 역전파와 경사하강법을 통해 파라미터를 업데이트 할 것이다.
- 문제는 심층 신경망의 각 계층들이 서로 어떻게 바뀔지 협의하에 상호작용하며 파라미터 업데이트가 이루어지는 것이 아니라는 것이다.
- i+1 계층은 i번째 계층의 결괏값을 바탕으로 더 나은 모델의 출력을 만들기 위해 파라미터 업데이트를 수행한다.
- 그런데 기껏 업데이트를 했더니 i번째 계층이 다른 출력을 반환하는 것이다.
- 신경망의 게층은 연쇄적으로 동작하기 때문에 공변량 문제가 발생할 수 있고 이것이 학습의 효율과 성능 개선을 저하시키는 문제로 지적되고 있다.

## 배치정규화의 동작
- 배치정규화는 미니배치 분포를 정규화하여 이러한 문제를 해결하고자 한다.
- 미니배치를 단위 가우시안분포로 바꾸는 정규표준분포화를 한 이후에 스케일 파라미터와 이동 파라미터를 적용한다.
- 이때 스케일파라미터와 이동파라미터는 학습되는 가중치 파라미터로써 신경망 내의 비선형적인 성질을 유지하도록 돕는다.


## 학습과 추론의 동작 차이
- 배치정규화 기법도 드롭아웃과 마찬가지로 학습과 추론에서의 동작 방식이 다르다.
- 배치정규화가 추론에서 동작이 다른 이유는 바로 미니배치의 평균과 표준편차를 구하는 방식 때문이다.
- 추론과정에서 미니배치의 전체 샘플들을 살펴보고 통계를 구하는 것은 올바르지 못하다.
- 따라서 추론과정에서는 들어오는 샘플들에 대한 이동평균과 이에 따른 표준편차를 계산하고 이를 활용하여 각 샘플들의 대략적인 표준정규분포화를 수행한다.

## 배치정규화의 구현
- 배치정규화 기법은 신경망 내부에서 계층으로 구현할 수 있다.
- 따라서 파이토치에서는 배치정규화 계층을 제공하고 있다.
- 그러면 배치정규화 계층을 적절한 위치에 삽입해야 한다.
- 원래 배치정규화를 제안한 사람은 선형계층과 활성함수 사이에 넣는것을 제안하였으나 이 경우에는 드롭아웃과 위치가 달라지게 되어 구현시에 무언가 불편해지는 상황이 연출되기도 한다.
- 따라서 많은 사람들이 배치정규화 계층을 원래 드롭아웃 계층을 삽입하던 위치에 삽입하곤한다. 이렇게 해더 배치정규화가 훌륭하게 동작한다.

In [1]:
import torch.nn as nn
net = nn.Sequential(
    nn.Linear(300,200),
    nn.LeakyReLU(),
    nn.BatchNorm1d(200),
    nn.Linear(200,100),
    nn.LeakyReLU(),
    nn.BatchNorm1d(100),
    nn.Linear(100,50),
    nn.LeakyReLU(),
    nn.BatchNorm1d(50),
    nn.Linear(50, 10)
)
net

Sequential(
  (0): Linear(in_features=300, out_features=200, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Linear(in_features=200, out_features=100, bias=True)
  (4): LeakyReLU(negative_slope=0.01)
  (5): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): Linear(in_features=100, out_features=50, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (9): Linear(in_features=50, out_features=10, bias=True)
)

- 배치정규화를 활용하게 되면 보통 드랍아웃을 쓰지 않는다.
- 필요에 따라 배치정규화와 드롭아웃중 정구화를 위한 계층을 선택하여 사용하기 바란다.

## 배치정규화의 장점과 한계
- 장점
    - 기존 정규화 기법과 달리 배치정규화의 경유 튜닝이 필요한 하이퍼파라미터가 추가되지 않는다는 점이 가장 큰 특징이자 장점이다.
    - 뿐만 아니라 정규화로 인한 일반화 성능 개선뿐만 아니라 학습과 수렴 속도가 빨라지게 된다.
- 한계
    - 단점은 거의 존재하지 않는 훌륭한 방법으로 사용할 수 있는 조건이라면 무조건 사용해야하는 정규화 기법이다.
    - 다만 동작 원리가 명확하게 밝혀지지 않았다.